In [1]:
import random

In [2]:
import itertools
import copy

In [3]:
class MagicMissile:
    def __init__(self):
        pass
    def cast(self, enemies, ava_spell_slots):
        spell_level = 1 + max([i for i,v in enumerate(ava_spell_slots) if v>0])
        missle = 2 + spell_level
        if not isinstance(enemies, list):
            enemies = [enemies] * missle
        for enemy in enemies:
            damage = random.randint(1, 4) + 1
            enemy.hp -= damage
        return spell_level

In [4]:
class Damage:
    def __init__(self, rolls, flat_bonus):
        self.rolls = rolls
        self.flat_bonus = flat_bonus
    def roll(self):
        rolls = sum([random.randint(1, roll) for roll in self.rolls])
        return rolls + self.flat_bonus

In [5]:
class Attack:
    def __init__(self, to_hit, damage):
        self.to_hit = to_hit
        self.damage = damage
    def apply_damage(self, to_hit, damage, enemy):
        if to_hit >= enemy.ac:
            enemy.hp -= damage
        
    def roll_hit(self, advantage = False, disadvantage = False, enemy=None):
        if advantage and disadvantage:
            advantage = disadvantage = False
        roll1 = self.to_hit + random.randint(1, 21)
        if advantage:
            roll2 = self.to_hit + random.randint(1, 21)
            self.apply_damage(max(roll1, roll2), self.damage(), enemy)
        if disadvantage:
            roll2 = self.to_hit + random.randint(1, 21)
            self.apply_damage(min(roll1, roll2), self.damage(), enemy)
        self.apply_damage(roll1, self.damage.roll(), enemy)

In [6]:
class Character:
    def __init__(self, hp, ac, attacks, stats, name, spells = [], spell_slots  = []):
        self.hp = hp
        self.max_hp = hp
        self.ac = ac
        self.spells = spells
        self.spell_slots = spell_slots
        self.max_spell_slots = copy.copy(spell_slots)
        if not isinstance(attacks, list):
            attacks = [attacks]
        self.attacks = attacks
        self.stats = stats
        self.name = name
    def roll_init(self):
        return self.stats['dex'] + random.randint(1, 21) + self.stats['dex']/100.
    def take_hit(self, to_hit, damage):
        if to_hit >= self.ac:
            self.hp -= damage

In [7]:
fighter = Character(
    hp = 13, ac = 18, attacks = Attack(to_hit = 4, damage = Damage(rolls = [6], flat_bonus = 2)), stats = {'dex':-1},
    name = 'fighter')
cleric = Character(
    hp = 10, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), stats = {'dex':-1},
    name = 'cleric')
warlock = Character(
    hp = 11, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)), stats = {'dex':2},
    name = 'warlock')
sorcerer = Character(
    hp = 9, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [10], flat_bonus = 0)),
    spells = [set([MagicMissile()]), ], spell_slots = [1,]
    , stats = {'dex':2},
    name = 'sorcerer')
rogue = Character(
    hp = 11, ac = 14, attacks = [
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)),
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 0)),
    ], stats = {'dex':3}, name = 'rogue')
team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [8]:
sorcerer.max_spell_slots

[1]

In [9]:
import numpy as np

In [10]:
def roll_init(x):
    init = [xi.roll_init() for xi in x]
    order = np.argsort(init)
    init_order = [x[i] for i in order[::-1]]
    return init_order
    

In [11]:
import pdb

In [12]:
def check_death(state):
    for char_group, return_val in [['npcs', 0], ['pcs', 1]]:
        state[char_group] = [c for c in state[char_group] if c.hp > 0]
        if len(state[char_group]) == 0:
            return return_val
    return -1

In [13]:
def combat(pcs, npcs):
    all_characters = pcs + npcs
    state = {'pcs': pcs, 'npcs': npcs}
    initiative = roll_init(all_characters)
    [setattr(c, 'hp', c.max_hp) for c in initiative]
    [setattr(c, 'spell_slots', c.max_spell_slots) for c in initiative]
    for _ in range(100):
        for character in initiative:
            start = len(state['pcs']) + len(state['npcs'])
            c = character
            if character in state['pcs']:
                enemy = state['npcs'][0]
            elif character in state['npcs']:
                enemy = state['pcs'][0]
            else:
                continue
            if sum(c.spell_slots) > 0:
                possible_spells = [spells for spells, slots in zip(c.spells, c.spell_slots) if slots > 0]
                
                spells = list(itertools.chain(*possible_spells))
                to_cast = spells[0]
                slot_used = to_cast.cast(enemy, c.spell_slots)
                c.spell_slots[slot_used-1] = c.spell_slots[slot_used-1]-1
               
                
            else:
                for attack in character.attacks:
                    attack.roll_hit(enemy = enemy)
                    end_val = check_death(state)
                    if end_val >= 0:
                        return end_val
            end_val = check_death(state)
            if end_val >= 0:
                return end_val
        initiative = [c for c in initiative if c.hp > 0]
    return -1

In [14]:
# sum([combat(team_PCs, team_NPCs) for _ in range(100)])/100

In [15]:
import copy

In [16]:

team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [19]:
cleric_team = [copy.deepcopy(cleric) for _ in range(4)]
rogue_team = [copy.deepcopy(rogue) for _ in range(4)]
sorc_team = [copy.deepcopy(sorcerer) for _ in range(4)]

sum([combat(cleric_team, sorc_team) for _ in range(1)])/100

[1]
[1]
[0] after
i cast it
[1]
[1]
[0] after
i cast it
[1]
[1]
[0] after
i cast it
[1]
[0]
[0]
[0]
[0]


0.0

In [18]:
sorc_team[0].max_spell_slots

[1]